# Preamble

### Imports

In [1]:
import os
# os.chdir("/users/sven/Documents/code/threepoint/python_scripts/")

# from file_loader import get_kappa_millennium
# from compute_aperture_mass import aperture_mass_computer
import numpy as np
import sys
import multiprocessing as mp
from itertools import permutations
import os
# from astropy.convolution import convolve_fft
from scipy import ndimage
from scipy.signal import fftconvolve,correlate
from astropy.io import fits
import warnings

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import quad,trapz
# import treecorr
from itertools import permutations
from tqdm.notebook import tqdm,trange
import sys
from matplotlib import colors
from math import log as mlog

import shutil



### General functions

In [2]:
def progressBar(name, value, endvalue, bar_length = 25, width = 20):
    """
    Displays a progress bar with "name", tracking the progess of value/endvalue
    """
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent*bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\r{0: <{1}} : [{2}]{3}%".format(name, width, arrow + spaces, int(round(percent*100))))
    sys.stdout.flush()
    if value == endvalue:
         sys.stdout.write('\n\n')


def Efunction(theta,L=60.):
    theta = theta/L
    if(hasattr(theta,"__len__")):
        result = np.zeros_like(theta)
        result[(theta<=1)] = (np.pi+ (theta[(theta<=1)]-4)*theta[(theta<=1)])/np.pi
        result[(theta>1)] = (2*((-1) + 2*np.sqrt(1 - theta[(theta>1)]**(-2))*theta[(theta>1)] \
                                - (theta[(theta>1)]**2)/2 - np.arccos(theta[(theta>1)]**(-1)) \
                                + np.arcsin(theta[(theta>1)]**(-1))))/np.pi
        result[(theta>=np.sqrt(2))] = 0
    else:
        if(theta<=1):
            result = (np.pi+ (theta-4)*theta)/np.pi
        if(theta>1):
            result = (2*((-1) + 2*np.sqrt(1 - theta**(-2))*theta \
                                - (theta**2)/2 - np.arccos(theta**(-1)) \
                                + np.arcsin(theta**(-1))))/np.pi
        if(theta>=np.sqrt(2)):
            result = 0
    return result

def index_of(theta_ap):
    return int(round(mlog(theta_ap,2)))-1


def standfracstr(s1,s2):
    x = s1+"-"+s2
    y = r"\sqrt{"+s1+"^2+"+s2+"^2}"
    return r"$\frac{"+x+"}{"+y+"}$"

def get_theta_combinations(theta_ap,return_str=True):
    n_ap = len(theta_ap)
    aperture_radii = []
    for i in range(n_ap):
        for j in range(i,n_ap):
            for k in range(j,n_ap):
                if return_str:
                    aperture_radii.append(r"$({}',{}',{}')$".format(theta_ap[i],theta_ap[j],theta_ap[k]))
    return aperture_radii

In [3]:
used_sim = "SLICS" #MS or SLICS

### define the correct L (if the calculated covariance is wrong check if you have done this)

In [4]:
if used_sim=="MS":
    L = (4*60-8*16)/60/180*np.pi
elif used_sim=="SLICS":
    L = (10*60-8*16)/60/180*np.pi
else:
    raise ValueError("Invalid used_sim. Allowed: MS,SLICS")

In [5]:
if(used_sim=="MS"):
    all_los = np.arange(64)
    fpath = "/vol/euclid2/euclid2_raid2/sven/mapCorr_for_T4/"
elif used_sim=="SLICS":
    all_los = np.arange(74,800)
    fpath = "/vol/euclid5raid5/data/euclid2_raid2/sven/mapCorr_SLICS_for_T4_kappa/"
else:
    raise ValueError("Invalid used_sim. Allowed: MS,SLICS")

    
n_los = len(all_los)

mask = np.zeros(n_los,dtype=bool)

    
all_theta_ap = [2,4,8,16]


n_theta_ap = len(all_theta_ap)
n_theta = 2000-1

results_mapsq_mapsq = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta_ap,n_theta_ap,n_theta))

mean_thetas = np.zeros(n_theta)


map_squared = np.zeros((n_los,n_theta_ap,n_theta_ap))

for x,los in enumerate(tqdm(all_los,desc="Reading map powers")):
    try:
        map_squared[x] = np.load(fpath+"map_squared_los_{}.npy".format(los))

        mask[x] = True #check which los are available, only read in that ones
        
    except Exception as e:
        pass
    
map_squared = map_squared[mask]



all_los = all_los[mask]
n_los = len(all_los)

results_mapsq_mapsq = np.zeros((n_los,n_theta_ap,n_theta_ap,n_theta))
mean_thetas = np.zeros(n_theta)

mask2 = np.ones(n_los,dtype=bool) #verify that everything is available




for x,los in enumerate(tqdm(all_los,desc="Reading mapsq mapsq")):
    for i in range(n_theta_ap):
        for j in range(i,n_theta_ap):
            for ii in range(n_theta_ap):
                for jj in range(ii,n_theta_ap):
                    try:
                        shutil.move(fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.dat.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los),
                                   fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    except Exception as e:
                        print("Could not move ","ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.dat.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    try:
                        gg = np.load(fpath+"ggcorr_mapsq_{}_{}_mapsq_{}_{}_los_{}.npy".format(all_theta_ap[i],
                                                    all_theta_ap[j],
                                                    all_theta_ap[ii],
                                                    all_theta_ap[jj],
                                                    los))
                    except Exception as e:
                        print(e)
                        mask2[x]=0
                        continue
                    for idi,idj in permutations([i,j]):
                        for idii,idjj in permutations([ii,jj]):
                            results_mapsq_mapsq[x,idi,idj,idii,idjj] = gg[1,:-1]



Reading map powers:   0%|          | 0/726 [00:00<?, ?it/s]

Reading mapsq mapsq: 0it [00:00, ?it/s]

In [6]:

map_squared_mean = np.mean(map_squared,axis=0)


if(np.sum(mask2)==len(mask2)):
    print("Length matches. computing average")

    results_mapsq_mapsq_mean = np.mean(results_mapsq_mapsq,axis=0)


else:
    print("WARNING: Length does not match: {}/{}. Excluding los with missing elements".format(np.sum(mask2),len(mask2)))

    results_mapsq_mapsq_mean = np.mean(results_mapsq_mapsq[mask2],axis=0)



Length matches. computing average


/home/laila/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/laila/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [7]:
thetas_mean = np.load(fpath+"theta_bins.npy")[:-1]




FileNotFoundError: [Errno 2] No such file or directory: '/vol/euclid5raid5/data/euclid2_raid2/sven/mapCorr_SLICS_for_T4_kappa/theta_bins.npy'

In [ ]:
def calculateNonGaussianMap2Covariance(th1,th2,L):
    idx1 = index_of(th1)
    idx2 = index_of(th2)

    mysum = results_mapsq_mapsq_mean[idx1,idx1,idx2,idx2]
    integrand = thetas_mean*Efunction(thetas_mean,L)*mysum
    result,err = trapz(integrand,thetas_mean)
    return 2*np.pi/(L*L)*result

In [ ]:
C=[]
for th1 in [4,8,16]:
    for th2 in [4,8,16]:
        C.append(calculateNonGaussianMap2Covariance(th1, th2, L))

C=np.array(C)

np.savetxt("./C_Map2_test.dat", C)